<a href="https://colab.research.google.com/github/francji1/01PR/blob/main/R/01PR_Ex07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01PR: Úvod do R - Cvičení 07

V tomto cvičení použijeme jeden z jenznámějších datasetů BostonHousing (možno získat z `library(mlbench)`  a `data(BostonHousing2)`).

Ukoly:

* Načtěte si data a vytvořte funkci, která pro libovolné dvě zadané numerické proměnné zobrazí jejich scatterplot, a velikost bodu bude odpovídat třetí numerické proměnné. Jako výchozí vezměte `cmdev`. Volitelně bude moci se přidat faktorová proměnná, která ovlivní barvu jednolivých bodů.

* Přidejte volitelnou proměnnou, která vykreslí do scatterplotu lienární odhad spolu s predikčním konfidenčním intervalem.

* Napište cyklus, který přes všechny rozumné numerické proměnné (`cmdev` jako výchozí target pro velikost bodů) a pro vybranou faktorovou proměnnou vykreslí předchozí obrázky a uloží je do souboru.

* Napište funkci, která vybranou proměnnou vykeslí do mapy podle souřadnic (`lon` a `lat`) a na vstupu bude numerická proměnná podle jejíž hodnoty  se bude vykreslovat intenzita barvy.

In [ ]:
install.packages("mlbench")
library(mlbench)

In [ ]:
data(BostonHousing2)

In [ ]:
? BostonHousing2

In [ ]:
summary(BostonHousing2)